In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# import

In [ ]:
!pip install -q git+https://github.com/MJAHMADEE/docs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from keras import backend as K
from keras.layers import Dense
from keras.models import Model

# prepare data

In [ ]:
! mkdir time-series
! unzip gdrive/MyDrive/DataDays2022/data/time-series.zip -d time-series

mkdir: cannot create directory ‘time-series’: File exists
Archive:  gdrive/MyDrive/DataDays2022/data/time-series.zip
  inflating: time-series/4.csv       
  inflating: time-series/5.csv       
  inflating: time-series/6.csv       
  inflating: time-series/7.csv       
  inflating: time-series/8.csv       
  inflating: time-series/9.csv       
  inflating: time-series/10.csv      
  inflating: time-series/11.csv      
  inflating: time-series/12.csv      
  inflating: time-series/13.csv      
  inflating: time-series/14.csv      
  inflating: time-series/15.csv      
  inflating: time-series/16.csv      
  inflating: time-series/17.csv      
  inflating: time-series/18.csv      
  inflating: time-series/19.csv      
  inflating: time-series/20.csv      
  inflating: time-series/21.csv      
  inflating: time-series/22.csv      
  inflating: time-series/23.csv      
  inflating: time-series/24.csv      
  inflating: time-series/25.csv      
  inflating: time-series/26.csv      
  inflati

In [ ]:
import os
X = None
Y = None
Yx = None
s = MinMaxScaler(feature_range=(-1,1))

for file in os.listdir("time-series"):
  if file.endswith(".csv"):
    print(file)
    file = '46.csv'
    train_df = pd.read_csv(os.path.join("time-series", file))

    time = train_df['timestamp']
    value = s.fit_transform(train_df[["value"]])
    print(value)
    label = train_df["label"]

    # reshape data
    # value = np.array(value).reshape(-1, 1)

    # preparing data for using LSTM model.
    window = 32
    if (X is not None) and (Y is not None) and (Yx is not None):
      X = np.concatenate([X, np.lib.stride_tricks.sliding_window_view(value.reshape(-1), window)])
      Yx = np.concatenate([Yx, value[window -1:]])
      Y = np.concatenate([Y, label[window -1:]])
    else:
      X = np.lib.stride_tricks.sliding_window_view(value.reshape(-1), window)
      Y = label[window -1:]
      Yx = value[window -1:]
    break

48.csv
[[-1.]
 [-1.]
 [-1.]
 ...
 [-1.]
 [-1.]
 [-1.]]


In [ ]:
np.min(value)

-1.0

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X.shape)
print(Y.shape)
print(Yx.shape)

(1510970, 32)
(1510970,)
(1510970, 1)


In [ ]:
# import os
# n=1
# for file in os.listdir("time-series"):
#   if file.endswith(".csv"):
#     file = '{}.csv'.format(n)
#     train_df = pd.read_csv(os.path.join("time-series", file))
#     time = train_df['timestamp']
#     value = train_df["value"]
#     label = train_df["label"]
#     print(file) 
#     # plot
#     plt.plot(time, value)
#     plt.plot(time,label)
#     # beautify the x-labels
#     plt.gcf().autofmt_xdate()
#     plt.show()
#     n+=1
#     if(n==81):
#       break

# metric functions

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# create model and primitive training 

In [ ]:
def get_model():
  model = keras.Sequential(
      [
        layers.Input((X.shape[1],1)),
        layers.LSTM(128),
        layers.RepeatVector(1),
        layers.Dropout(0.1),
        layers.LSTM(64),
        layers.Dropout(0.1),
        layers.RepeatVector(1),
        layers.LSTM(32),
        # layers.Dense(32, activation='relu'),
        # layers.Dense(1, activation='sigmoid')
        layers.Dense(1)
      ]
  )
  return model

In [ ]:
model = get_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 128)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 64)            0         
 or)                                                    

In [ ]:
model.compile(
    loss=keras.losses.mean_squared_error,
    optimizer="adam",
    # loss='binary_crossentropy',
    # metrics=['acc',f1_m,precision_m, recall_m]
)

history = {}
history['model'] = model.fit(
    X, Yx, validation_split=0.2, batch_size=64, epochs=30,
    callbacks=[early_stopping]
)

Epoch 1/30
18888/18888 [==============================] - 176s 9ms/step - loss: 6.4009e-04 - val_loss: 1.2586e-05
Epoch 2/30
18888/18888 [==============================] - 169s 9ms/step - loss: 1.1660e-04 - val_loss: 1.3748e-05
Epoch 3/30
18888/18888 [==============================] - 162s 9ms/step - loss: 8.0982e-05 - val_loss: 6.3329e-06
Epoch 4/30
18888/18888 [==============================] - 168s 9ms/step - loss: 6.4307e-05 - val_loss: 2.1830e-05
Epoch 5/30
18888/18888 [==============================] - 161s 9ms/step - loss: 5.4961e-05 - val_loss: 1.3805e-05
Epoch 6/30
18888/18888 [==============================] - 163s 9ms/step - loss: 4.9041e-05 - val_loss: 7.0682e-06
Epoch 7/30
18888/18888 [==============================] - 165s 9ms/step - loss: 4.7231e-05 - val_loss: 2.9012e-06
Epoch 8/30
18888/18888 [==============================] - 172s 9ms/step - loss: 4.2530e-05 - val_loss: 4.8597e-06
Epoch 9/30
18888/18888 [==============================] - 174s 9ms/step - loss: 4.0457e-

In [ ]:
model.save("gdrive/MyDrive/DataDays2022/pretrained_model.h")

In [ ]:
from keras.models import load_model
model = load_model('gdrive/MyDrive/DataDays2022/pretrained_model.h')

# change last layers of pretrained_model

In [ ]:
# freez all layers
for layer in model.layers[:]:
  layer.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 128)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 64)            0         
 or)                                                    

In [ ]:
model.pop()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 128)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 64)            0         
 or)                                                    

In [ ]:
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               66560     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 128)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 64)            0         
 or)                                                    

# prepare dataset with equal number of sample of each class



In [ ]:
bool_train_labels = Y_train != 0

In [ ]:
pos_features = X_train[bool_train_labels]
neg_features = X_train[~bool_train_labels]

pos_labels = Y_train[bool_train_labels]
neg_labels = Y_train[~bool_train_labels]

In [ ]:
print(len(neg_features), len(pos_features))

1185610 23166


In [ ]:
neg_features, neg_labels = shuffle(neg_features, neg_labels)

neg_features = neg_features[:len(pos_features)]
neg_labels = neg_labels[:len(pos_labels)]

In [ ]:
X_train2 = np.concatenate([pos_features, neg_features])
Y_train2 = np.concatenate([pos_labels, neg_labels])
X_train2, Y_train2 = shuffle(X_train2, Y_train2)

In [ ]:
# class_weight = {0: 1, 1: 2}
model.compile(
    loss=keras.losses.mean_squared_error,
    optimizer="adam",
    # loss='binary_crossentropy',
    metrics=['acc',f1_m,precision_m, recall_m]
)

history = {}
history['model'] = model.fit(
    X_train2, Y_train2, validation_split=0.2, batch_size=64, epochs=50,
    callbacks=[early_stopping],
    # class_weight=class_weight
)

Epoch 1/50
580/580 [==============================] - 11s 9ms/step - loss: 0.2232 - acc: 0.6626 - f1_m: 0.6271 - precision_m: 0.6982 - recall_m: 0.5793 - val_loss: 0.2163 - val_acc: 0.6706 - val_f1_m: 0.6572 - val_precision_m: 0.6876 - val_recall_m: 0.6362
Epoch 2/50
580/580 [==============================] - 4s 7ms/step - loss: 0.2147 - acc: 0.6756 - f1_m: 0.6606 - precision_m: 0.6853 - recall_m: 0.6449 - val_loss: 0.2126 - val_acc: 0.6770 - val_f1_m: 0.6711 - val_precision_m: 0.6870 - val_recall_m: 0.6628
Epoch 3/50
580/580 [==============================] - 4s 7ms/step - loss: 0.2116 - acc: 0.6800 - f1_m: 0.6683 - precision_m: 0.6868 - recall_m: 0.6578 - val_loss: 0.2101 - val_acc: 0.6862 - val_f1_m: 0.6785 - val_precision_m: 0.6979 - val_recall_m: 0.6671
Epoch 4/50
580/580 [==============================] - 4s 7ms/step - loss: 0.2087 - acc: 0.6858 - f1_m: 0.6789 - precision_m: 0.6882 - recall_m: 0.6769 - val_loss: 0.2074 - val_acc: 0.6882 - val_f1_m: 0.6845 - val_precision_m: 0.695

In [ ]:
model.save("gdrive/MyDrive/DataDays2022/final_model.h")

In [ ]:
from keras.models import load_model
model = load_model('gdrive/MyDrive/DataDays2022/final_model.h')

ValueError: ignored

#evaluate final model with test data

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=128)

2361/2361 [==============================] - 12s 5ms/step - loss: 0.2452 - acc: 0.7624 - f1_m: 0.0786 - precision_m: 0.0436 - recall_m: 0.5131


In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=128)

2361/2361 [==============================] - 12s 5ms/step - loss: 0.2273 - acc: 0.7147 - f1_m: 0.0738 - precision_m: 0.0402 - recall_m: 0.5779


In [ ]:
import os
import numpy as np
import pandas as pd

input_path = 'time-series'
output_path = '/output'


class TimeSeriesAnomalyDetector:
  def __call__(self, df):
    signal = np.array(df['value'])
    n = len(signal)
    mean = np.mean(signal)
    std = np.std(signal)
    label = (signal - mean > 2 * std).astype(np.int32)
    df['p-label'] = label
    return df
    
anomaly_detector = TimeSeriesAnomalyDetector()

y_true = np.array([])
y_pred = np.array([])
if __name__ == '__main__':
  for filename in os.listdir(input_path):
    input_df = pd.read_csv(os.path.join(input_path, filename))
    # print(input_path, len(input_df))
    result = anomaly_detector(input_df)
    # print(result)
    y_true = np.concatenate([y_true, np.array(result['label']).astype("float32")])
    y_pred = np.concatenate([y_pred, np.array(result['p-label']).astype("float32")])
    # y_true = np.array(result['label']).astype("float32")
    # y_pred = np.array(result['p-label']).astype("float32")
    # print(y_pred)
  print('f1_m' ,f1_m(y_true, y_pred))
  print('precision_m' ,precision_m(y_true, y_pred))
  print('recall_m' ,recall_m(y_true, y_pred))
  # print(np.array(result['label']))
    # result.to_csv(os.path.join(output_path, filename))
    # print(f'item {filename} processed.')

f1_m tf.Tensor(0.2037837662774691, shape=(), dtype=float64)
precision_m tf.Tensor(0.3228405753420977, shape=(), dtype=float64)
recall_m tf.Tensor(0.14888001389078354, shape=(), dtype=float64)


#OR between Model and Statistics method.

In [ ]:
class ModelTimeSeriesAnomalyDetector:
  def __call__(self, df, model):
    s = MinMaxScaler(feature_range=(-1,1))
    value = s.fit_transform(df[["value"]])
    x = np.lib.stride_tricks.sliding_window_view(value.reshape(-1), window)
    model = model
    label = np.zeros(31).reshape(-1,1)
    label = np.concatenate([label,(model.predict(x) > 0.5).astype("int32")])
    label = np.reshape(label, len(label))
    df['p-label'] = label    
    return df

  def creat_windows(self,value):
    X = None
    window = 32
    if (X is not None):
      X = np.concatenate([X, np.lib.stride_tricks.sliding_window_view(value.reshape(-1), window)])
    else:
      X = np.lib.stride_tricks.sliding_window_view(value.reshape(-1), window)
    return X

  def recall_m(self,y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

  def precision_m(self,y_true, y_pred):
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
      precision = true_positives / (predicted_positives + K.epsilon())
      return precision

  def f1_m(self,y_true, y_pred):
      precision = self.precision_m(y_true, y_pred)
      recall = self.recall_m(y_true, y_pred)
      return 2*((precision*recall)/(precision+recall+K.epsilon()))

  def create_model(self,X):
    model = keras.Sequential(
        [
          layers.Input((X.shape[1],1)),
          layers.LSTM(128),
          layers.RepeatVector(1),
          layers.Dropout(0.5),
          layers.LSTM(64),
          layers.Dropout(0.5),
          layers.RepeatVector(1),
          layers.LSTM(32),
          layers.Dense(32, activation='relu'),
          layers.Dense(1, activation='sigmoid')
        ]
    )
    return model

In [ ]:
anomaly_detector = TimeSeriesAnomalyDetector()
model_anomaly_detector = ModelTimeSeriesAnomalyDetector()

y_true = np.array([])
y_pred_s = np.array([])
y_pred_m = np.array([])
if __name__ == '__main__':
  for filename in os.listdir(input_path):
    input_df = pd.read_csv(os.path.join(input_path, filename))
    # print(input_path, len(input_df))
    result = anomaly_detector(input_df)
    model_result = model_anomaly_detector(input_df, model)
    print('model_result', model_result)
    y_true = np.concatenate([y_true, np.array(result['label']).astype("float32")[31:]])
    y_pred_s = np.concatenate([y_pred_s, np.array(result['p-label']).astype("float32")[31:]])
    # print(y_pred_s.shape)
    y_pred_m = np.concatenate([y_pred_m, np.array(model_result['p-label']).astype("float32")[31:]])
    print(len(y_true), len(y_pred_s), len(y_pred_m))
    break
    # y_true = np.array(result['label']).astype("float32")
    # y_pred = np.array(result['p-label']).astype("float32")
    # print(y_pred)
  # print('f1_m' ,f1_m(y_true, y_pred))
  # print('precision_m' ,precision_m(y_true, y_pred))
  # print('recall_m' ,recall_m(y_true, y_pred))

model_result          timestamp  value  label  p-label
0       1491041280  31.30      0      0.0
1       1491041340  32.20      0      0.0
2       1491041400  33.06      0      0.0
3       1491041460  32.35      0      0.0
4       1491041520  32.69      0      0.0
...            ...    ...    ...      ...
146250  1500012600  40.51      0      0.0
146251  1500012660  41.61      0      0.0
146252  1500012720  41.43      0      0.0
146253  1500012780  43.00      0      0.0
146254  1500012840  41.98      0      0.0

[146255 rows x 4 columns]
146224 146224 146224


In [ ]:
print('f1_m' ,f1_m(y_true, y_pred_s))
print('precision_m' ,precision_m(y_true, y_pred_s))
print('recall_m' ,recall_m(y_true, y_pred_s))

f1_m tf.Tensor(0.6192337332454712, shape=(), dtype=float64)
precision_m tf.Tensor(0.9875311718235582, shape=(), dtype=float64)
recall_m tf.Tensor(0.4510250568962386, shape=(), dtype=float64)


In [ ]:
print('f1_m' ,f1_m(y_true, y_pred_m))
print('precision_m' ,precision_m(y_true, y_pred_m))
print('recall_m' ,recall_m(y_true, y_pred_m))

f1_m tf.Tensor(0.6192337332454712, shape=(), dtype=float64)
precision_m tf.Tensor(0.9875311718235582, shape=(), dtype=float64)
recall_m tf.Tensor(0.4510250568962386, shape=(), dtype=float64)


In [ ]:
print(len(Y), len(y_pred))

1510970 1513481
